###Ingestion del archivo "movie_genre.json"

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/commom_functions"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

####Paso 1- Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
movie_genre_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("genreId", IntegerType(), True)
])

In [0]:
movie_genre_df = spark.read \
                .schema(movie_genre_schema) \
                .json(f"{bronze_folder_path}/{v_file_date}/movie_genre.json")

In [0]:
movie_genre_df.printSchema()

In [0]:
display(movie_genre_df)

####Paso 2 - Renombrar las columnas y añadir nuevas columnas
- "movieId" renombar a "movie_id" y "genreId" a "genre_id"
- Agregar las columnas "ingestion_date" y "environment"

In [0]:
from pyspark.sql.functions import col, current_timestamp, lit

In [0]:
movie_genre_with_columns_df = add_ingestion_dat(movie_genre_df) \
                            .withColumnRenamed("movieId","movie_id") \
                            .withColumnRenamed("genreId","genre_id") \
                            .withColumn("environment", lit(v_environment)) \
                            .withColumn("file_date",lit(v_file_date))
display(movie_genre_with_columns_df)

####Paso 3 - Escribir la salida en un formato "Parquet"
- Hacer particionado en silver por la columna movie_id

In [0]:
#movie_genre_with_columns_df.write.mode("overwrite").parquet(f"{silver_folder_path}/movie_genre")
#movie_genre_with_columns_df.write.mode("append").partitionBy("file_date").format("parquet").saveAsTable("movie_silver.movie_genre")
merge_condition='tgt.movie_id = src.movie_id and tgt.genre_id = src.genre_id and tgt.file_date = src.file_date'
merge_delta_lake(movie_genre_with_columns_df, "movie_silver", "movies_genre", silver_folder_path, merge_condition, "file_date")

In [0]:
movie_genre_with_columns_df.write.mode("append").partitionBy("file_date").parquet(f"{silver_folder_path}//movie_genre")

In [0]:
display(spark.read.format("delta").load("/mnt/mymoviehistory/silver/movies"))

In [0]:
%sql
select file_date, count(1) 
from movie_silver.movies_genre
group by file_date;

In [0]:
dbutils.notebook.exit("Success")